<a href="https://colab.research.google.com/github/mortgad/DLVR/blob/main/Regression_approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Connect to google

In [1]:
from google.colab import drive
drive.mount('/content/drive')

# save_model_path = '/content/drive/MyDrive/DL for VR/Project/Models' # Thomas
# save_model_path = '/content/drive/MyDrive/Deep_Learning_Visual_Recognition/Project/' # Morten
save_model_path = '/content/drive/MyDrive/Visual reg' # Mads


Mounted at /content/drive


In [2]:
!git clone https://github.com/mortgad/DLVR.git
%cd DLVR

Cloning into 'DLVR'...
remote: Enumerating objects: 340, done.
remote: Counting objects: 100% (198/198), done.
remote: Compressing objects: 100% (102/102), done.
remote: Total 340 (delta 118), reused 163 (delta 95), pack-reused 142 (from 1)
Receiving objects: 100% (340/340), 16.85 MiB | 37.43 MiB/s, done.
Resolving deltas: 100% (180/180), done.
/content/DLVR


In [4]:
from utils.utkface import load_utkface
load_utkface()

from utils.utkface import preprocess_utkface
df_utkface, df_utkface_raw = preprocess_utkface()

Extracted part3.tar.gz
Extracted part2.tar.gz
Extracted part1.tar.gz
Deleted: /content/extracted/part3/44_1_4_20170116235150272.pg
Deleted: /content/extracted/part3/55_0_0_20170116232725357jpg
Deleted: /content/extracted/part3/.DS_Store
Deleted: /content/extracted/part1/61_1_20170109142408075.jpg
Deleted: /content/extracted/part1/61_3_20170109150557335.jpg
Deleted: /content/extracted/part2/53__0_20170116184028385.jpg
Deleted: /content/extracted/part2/39_1_20170116174525125.jpg
Deleted: /content/extracted/part3/24_0_1_20170116220224657 .jpg
Processing from: /content/extracted/part1
Processing from: /content/extracted/part2
Processing from: /content/extracted/part3


# Get data for modelling


In [8]:
# Drop not needed colums:
df = df_utkface_raw[['age_raw','gender','race','file']]

df = df[df['age_raw']<=100]

df.rename(columns={'age_raw': 'age_code'}, inplace=True)
df.rename(columns={'gender': 'gender_code'}, inplace=True)
df.rename(columns={'race': 'race_code'}, inplace=True)

In [10]:
from utils.preprocessing import create_lists

images, ages, races, genders = create_lists(df)
print(f"Loaded {len(images)} images.")

# Number of classes for target variable
num_classes_age = len(set(ages))
num_classes_gender = len(set(genders))
num_classes_ethnicity = len(set(races))
print(f"Number of age classes: {num_classes_age}")
print(f"Number of gender classes: {num_classes_gender}")
print(f"Number of ethnicity classes: {num_classes_ethnicity}")


Loaded 24079 images.
Number of age classes: 97
Number of gender classes: 2
Number of ethnicity classes: 5


In [15]:
from sklearn.model_selection import train_test_split
from keras.applications.resnet_v2 import preprocess_input

images_preprocessed = preprocess_input(images)

# Split into train (80%) and temp (20%)
X_train, X_temp, y_train_age, y_temp_age, y_train_races, y_temp_races, y_train_gender, y_temp_gender = train_test_split(images_preprocessed, ages, races, genders, test_size=0.20, random_state=42, stratify=ages)

# Split temp into validation (75% of temp, which is 15% of original data) and test (25% of temp, which is 5% of original data)
X_val, X_test, y_val_age, y_test_age, y_val_races, y_test_races, y_val_gender, y_test_gender = train_test_split(X_temp, y_temp_age, y_temp_races, y_temp_gender, test_size=0.25, random_state=42)


print(f"Training set: {X_train.shape}, Training labels: {y_train_age.shape}")
print(f"Age validation set: {X_val.shape}, Age validation labels: {y_val_age.shape}")
print(f"Age test set: {X_test.shape}, Age test labels: {y_test_age.shape}")
print(f"Race validation labels: {y_val_races.shape}")
print(f"Race test labels: {y_test_races.shape}")
print(f"Gender validation labels: {y_val_gender.shape}")
print(f"Gender test labels: {y_test_gender.shape}")

Training set: (19263, 224, 224, 3), Training labels: (19263,)
Age validation set: (3612, 224, 224, 3), Age validation labels: (3612,)
Age test set: (1204, 224, 224, 3), Age test labels: (1204,)
Race validation labels: (3612,)
Race test labels: (1204,)
Gender validation labels: (3612,)
Gender test labels: (1204,)


# Training model

In [16]:
from tensorflow.keras.applications import ResNet50V2
from tensorflow import keras

# Now proceed with the ResNet50V2 model using the preprocessed images
conv_base = ResNet50V2(
    include_top=False,  # Exclude the top fully connected layers
    weights="imagenet",  # Load pre-trained ImageNet weights
    input_shape=(images_preprocessed[0].shape[0], images_preprocessed[0].shape[1], images_preprocessed[0].shape[2]),  # Input shape from the preprocessed images
)

conv_base.trainable = False  # Freeze the pre-trained layers

print("Conv base, training, and validation sets have been created...")

# Create the new model using ResNet50V2 as the base
inputs = keras.Input(shape=(images_preprocessed[0].shape[0], images_preprocessed[0].shape[1], images_preprocessed[0].shape[2]))
from tensorflow.keras import layers

x = conv_base(inputs)
x = layers.GlobalAveragePooling2D()(x)
x = layers.BatchNormalization()(x)

x = layers.Dense(512, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.3)(x)  # Dropout with a 50% rate

x = layers.Dense(256, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.2)(x)  # Dropout with a 40% rate

x = layers.Dense(128, activation='relu')(x)
x = layers.BatchNormalization()(x)

x = layers.Dense(64, activation='relu')(x)

# Adjust the output layer for age prediction
outputs = layers.Dense(1, activation='linear')(x)  # Single neuron for numeric output

model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()


94668760/94668760 [==============================] - 0s 0us/step
Conv base, training, and validation sets have been created...
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 resnet50v2 (Functional)     (None, 7, 7, 2048)        23564800  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 batch_normalization (Batch  (None, 2048)              8192      
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 512)               1049088   


In [ ]:
# Compile the model
optimizer = keras.optimizers.Adam(learning_rate=1.5e-4)
model.compile(loss="mean_squared_error",  # Use MSE or MAE for regression
              optimizer=optimizer,
              metrics=["mae"])  # Optionally use MAE as a metric

# Define the callback for saving the best model
from tensorflow.keras.callbacks import ModelCheckpoint
file_path = '/content/drive/MyDrive/Visual reg' + "/"
model_name = 'resnet50v2_model_baseline.keras'

# Save the model in the new Keras format with the best validation accuracy
checkpoint = ModelCheckpoint(file_path + model_name,
                             monitor='val_mae',  # Monitor MAE for regression
                             verbose=1,
                             save_best_only=True,
                             mode='min')  # For MAE, lower is better

# Train the model
resnet_history = model.fit(
    X_train,
    y_train_age,
    epochs=20,
    batch_size=32,
    validation_data=(X_val, y_val_age),
    callbacks=[checkpoint]
)

Epoch 1/20
602/602 [==============================] - ETA: 0s - loss: 686.0565 - mae: 20.6587
Epoch 1: val_mae improved from inf to 12.23808, saving model to /content/drive/MyDrive/Visual reg/resnet50v2_model_baseline.keras
602/602 [==============================] - 286s 469ms/step - loss: 686.0565 - mae: 20.6587 - val_loss: 242.5660 - val_mae: 12.2381
Epoch 2/20
509/602 [========================>.....] - ETA: 36s - loss: 197.4240 - mae: 10.7164

In [ ]:
from utils.preprocessing import evaluate_and_plot

# Evaluate and plot the model
evaluate_and_plot(resnet_history)